# Метод Гаусса

In [1]:
import numpy as np
import scipy.integrate as integrate
from scipy.special import roots_legendre

Задача: посчитать интеграл $\int_0^2 \frac{2+ cos(1+ \sqrt x) e^{0.5x}}{\sqrt{1+0.5x}}dx$. Используем метод Гаусса. Для того чтобы можно было применить полином Лежандра, нужно чтобы пределы интегрирования были от -1 до 1.

In [2]:
def f_initial(x):
    return (2 + np.cos(1 + np.sqrt(x)))*np.exp(0.5*(x)) / np.sqrt(1 + 0.5*x)
#Меняем пределы интегрирования на -1, 1
def func(initial, a, b, x):
    return initial(( ( b - a ) / 2 ) * ( x + 1 + 2*a / (b - a) ) ) * ( b - a ) / 2

С помощью библиотечной функции roots_legendre считаем корни и веса в узлах. f - интегрируемая функция с пределами интегрирования от -1 до 1, n - количество узлов. Сам интеграл считаем как сумму значений func(roots), помноженных на веса 

In [3]:
def GL(f,n):
    roots, weights = roots_legendre(n)
    return np.sum(func(f,0,2,roots)*weights)

Посчитаем количество узлов в зависимости от заданной точности

In [4]:
def order(f, eps):
    prev = 0
    n = 0
    while(True):
        n += 1
        new = GL(f, n)
        if np.abs(prev - new)<eps:
            break
        prev = new
    eps_degree = np.round(-np.log10(eps))
    new = np.round(new, int(eps_degree))
    return new, n

Посмотрим результат при eps =$ 10^{-10}$

In [5]:
result=order(f_initial, 1e-10)
print('result =', result[0], '\nn =', result[1])

result = 4.423368482 
n = 293


Другие eps для примера

In [6]:
print('result =', order(f_initial, 1e-4)[0], '\nn =', order(f_initial, 1e-4)[1])
print('result =', order(f_initial, 1e-7)[0], '\nn =', order(f_initial, 1e-7)[1])
print('result =', order(f_initial, 1e-9)[0], '\nn =', order(f_initial, 1e-9)[1])

result = 4.4232 
n = 10
result = 4.4233669 
n = 53
result = 4.423368438 
n = 165


Сравним наш ответ при eps =$ 10^{-10}$ и результат работы integrate.quad

In [7]:
true_result = integrate.quad(lambda x: (2+np.cos(1 + np.sqrt(x)))*np.exp(0.5*(x)) /np.sqrt(1 + 0.5*x) , 0, 2)
eps = 1
while(True):
    if np.allclose(result[0], true_result[0], rtol = eps, atol = 0) == True:
        print('eps =', eps, ' - Match')
    else:
        print('eps =', eps, ' - Mismatch;\n Quad: ', np.round(true_result[0], int(-np.log10(eps))),
              '\n Gauss-Legendre: ', np.round(result[0], int(-np.log10(eps))))
        break    
    eps = eps/10

eps = 1  - Match
eps = 0.1  - Match
eps = 0.01  - Match
eps = 0.001  - Match
eps = 0.0001  - Match
eps = 1e-05  - Match
eps = 1.0000000000000002e-06  - Match
eps = 1.0000000000000002e-07  - Match
eps = 1.0000000000000002e-08  - Match
eps = 1.0000000000000003e-09  - Mismatch;
 Quad:  4.423368492 
 Gauss-Legendre:  4.423368482


Видно, что различия появляются при eps =$ 10^{-9}$. Стоит помнить, что приближение $\int_{-1}^1 W(x)f(x)dx \approx \sum_{j=1}^N w_j f(x_j)$, которым мы воспользовались, является точным, если f(x) - полином.